In [1]:
%matplotlib inline

import os
import pickle
import numpy as np

import gym
import neat
import cv2

import utils.PyPlotReporter
import utils.CSVReporter
from utils.EnvEvaluator import * 
from utils.CPPNPythonNeat import PythonNeatCPPN
from utils.CPPNPyTorch import PyTorchCPPN
from utils.SubstrateCNN import *
from utils.Profiling import get_exec_times_pd, profile_section
from utils.Atari import CONTROLLER_TO_ACTION,CONTROLLER_TO_ACTION_SHORT, CONTROLLER_TO_ACTION_FORCE
import torchvision.transforms as transforms


In [2]:
class MyEvaluator(HyperNeatEnvEvaluator):

    def make_cppn(self, genome: DefaultGenome, config: Config):
   #     return PythonNeatCPPN(genome, config)
        return PyTorchCPPN(genome, config, batch_size=64)

    def make_substrate(self):
        return SubstrateCNN(nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 16, 3),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 16, 3),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 128, (3, 2)),
            nn.Sigmoid(),
            nn.Conv2d(128, 3, 1),
            nn.Sigmoid(),
        ))

    def preproc_img(self, observation):
        process = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
       # grey = cv2.cvtColor(observation, cv2.COLOR_RGB2GRAY)
        small = cv2.resize(observation, dsize=(32, 42), interpolation=cv2.INTER_AREA)
        return process(small).unsqueeze(0)

    def activate_net(self, sub, observation):
        with profile_section("preproc"):
            grey = self.preproc_img(observation)
        with profile_section("sub_forward"):
            activation = sub.forward(grey)[0,:,0,0].numpy()
        out = tuple(activation > .5)
        action = CONTROLLER_TO_ACTION_FORCE[out]
        return action
    
evaluator = MyEvaluator("Riverraid-v0", 2000, n_workers=1, seed=1111)

In [3]:
evaluator.make_substrate().count_params()

[448, 0, 0, 2320, 0, 0, 2320, 0, 0, 12416, 0, 387, 0]

In [4]:
config_path = "./configs/hyperataricnn.cfg" 

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

In [5]:
pop = neat.Population(config)
pop.add_reporter(utils.CSVReporter.CSVReporter("saves/progress.csv"))
pop.add_reporter(utils.PyPlotReporter.PyPlotReporter())
pop.add_reporter(neat.Checkpointer(1, None, "saves/cp_atari_6_"))

In [ ]:
gnome = pop.run(evaluator.eval_all_genomes, 100)

In [8]:
get_exec_times_pd()

,name,avg_time,total_time,N
0,make_cppn,0.000353,0.004947,14
1,make_substrate,0.000762,0.010673,14
2,get_weights,0.007479,0.523529,70
3,get_biases,0.000217,0.015163,70
4,make_net,0.040638,0.568930,14
5,env_make,0.016598,0.232376,14
6,preproc,0.000135,0.520739,3864
7,sub_forward,0.000464,1.793890,3864
8,env_step,0.000489,1.890030,3864


In [ ]:
name 	        avg_time 	total_time 	N
# NeatPython ~17000 weights
get_weights 	0.970316 	24.257888 	25
# Pytorch 
get_weights 	0.008045 	0.201118 	25
